In [4]:
import sys  
sys.path.insert(0, '/Users/pratikaher/DGL/GRecSy/')

In [5]:
from random import sample
import dgl
import torch
import torch.nn.functional as F
import pandas as pd
from utils import set_random_seed, collate_molgraphs, load_model, collate_movie_graphs, train_test_split_by_time, _split_data
import numpy as np
import torch
import torch.nn as nn
import argparse
from configure import get_exp_configure
import scipy.sparse as sp
from torch.utils.data import DataLoader

from dgllife.utils import EarlyStopping, Meter
from model import compute_loss

In [6]:
df_rating = pd.read_csv('/Users/pratikaher/DGL/graph-rec/ml-1m/ratings.dat', sep='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'],  engine='python')
df_user = pd.read_csv('/Users/pratikaher/DGL/graph-rec/ml-1m/users.dat', sep='::', header=None, names=['user_id', 'gender', 'age', 'occupation', 'zipcode'],  engine='python')
df_movie = pd.read_csv('/Users/pratikaher/DGL/graph-rec/ml-1m/movies.dat', sep='::', header=None, names=['movie_id', 'title', 'genre'],  engine='python')

In [7]:
df_temp = df_rating.merge(df_movie, left_on='movie_id', right_on='movie_id', how='left')
df_final = df_temp.merge(df_user, left_on='user_id', right_on='user_id', how='left')

In [10]:
df_final = df_final[["user_id","movie_id","rating","age"]]

In [12]:
df_final.tail()

,user_id,movie_id,rating,age
1000204,6040,1091,1,25
1000205,6040,1094,5,25
1000206,6040,562,5,25
1000207,6040,1096,4,25
1000208,6040,1097,4,25


In [13]:
g = dgl.graph((df_final['user_id'].to_numpy(), df_final['movie_id'].to_numpy()))

In [15]:
isolated_nodes = ((g.in_degrees() == 0) & (g.out_degrees() == 0)).nonzero().squeeze(1)
g = dgl.remove_nodes(g, isolated_nodes)

In [17]:
g.edata['rating'] = torch.unsqueeze(torch.tensor(df_final['rating'].to_numpy()), dim = 1)
g.ndata['age'] = torch.unsqueeze(torch.tensor(df_user['age'].to_numpy()).float() / 100, dim = 1)

In [18]:
g

Graph(num_nodes=6040, num_edges=1000209,
      ndata_schemes={'age': Scheme(shape=(1,), dtype=torch.float32)}
      edata_schemes={'rating': Scheme(shape=(1,), dtype=torch.int64)})

In [24]:
u, v = g.edges()

In [25]:
eids = np.arange(g.number_of_edges())
eids = np.random.permutation(eids)
test_size = int(len(eids) * 0.1)
train_size = g.number_of_edges() - test_size

In [27]:
test_size, train_size

(100020, 900189)

In [28]:
test_pos_u, test_pos_v = u[eids[:test_size]], v[eids[:test_size]]
train_pos_u, train_pos_v = u[eids[test_size:]], v[eids[test_size:]]

In [30]:
# # Find all negative edges and split them for training and testing
# adj = sp.coo_matrix((np.ones(len(u)), (u.numpy(), v.numpy())))
# adj_neg = 1 - adj.todense() - np.eye(adj.shape[0], adj.shape[1])
# neg_u, neg_v = np.where(adj_neg != 0)

# neg_eids = np.random.choice(len(neg_u), g.number_of_edges() // 2)
# test_neg_u, test_neg_v = neg_u[neg_eids[:test_size]], neg_v[neg_eids[:test_size]]
# train_neg_u, train_neg_v = neg_u[neg_eids[test_size:]], neg_v[neg_eids[test_size:]]


In [31]:
train_g = dgl.remove_edges(g, eids[:test_size])

In [32]:
# test_g = dgl.remove_edges(g, eids[test_size:])

In [34]:
train_pos_g = dgl.graph((train_pos_u, train_pos_v), num_nodes=g.number_of_nodes())
# train_neg_g = dgl.graph((train_neg_u, train_neg_v), num_nodes=g.number_of_nodes())

test_pos_g = dgl.graph((test_pos_u, test_pos_v), num_nodes=g.number_of_nodes())
# test_neg_g = dgl.graph((test_neg_u, test_neg_v), num_nodes=g.number_of_nodes())

In [35]:
train_pos_g.edata['rating'] = torch.unsqueeze(torch.tensor(df_final['rating'].to_numpy()[:train_size]), dim = 1)

train_pos_g.ndata['age'] = torch.unsqueeze(torch.tensor(df_user['age'].to_numpy()[:train_size]).float()/100, dim = 1)

In [36]:
test_pos_g.edata['rating'] = torch.unsqueeze(torch.tensor(df_final['rating'].to_numpy()[:test_size]), dim = 1)

test_pos_g.ndata['age'] = torch.unsqueeze(torch.tensor(df_user['age'].to_numpy()[:test_size]).float()/100, dim = 1)

In [44]:
sampler = dgl.dataloading.MultiLayerNeighborSampler([15, 10, 5])
dataloader = dgl.dataloading.EdgeDataLoader(
        train_g, train_g.nodes(), 
        sampler, negative_sampler=dgl.dataloading.negative_sampler.Uniform(5), 
        shuffle=True, drop_last=False,
        batch_size=16
        )

In [45]:
sum_values = 0
for _, pos_g, neg_g, blocks in dataloader:
    print(pos_g, neg_g)
    break
    sum_values += (len(blocks[0].srcdata['_ID'].tolist()))

print(sum_values)

Graph(num_nodes=109, num_edges=16,
      ndata_schemes={'age': Scheme(shape=(1,), dtype=torch.float32), '_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={'rating': Scheme(shape=(1,), dtype=torch.int64), '_ID': Scheme(shape=(), dtype=torch.int64)}) Graph(num_nodes=109, num_edges=80,
      ndata_schemes={'_ID': Scheme(shape=(), dtype=torch.int64)}
      edata_schemes={})
0


In [ ]:
sampler = dgl.dataloading.MultiLayerFullNeighborSampler(3)
nodeloader_test = dgl.dataloading.NodeDataLoader(
        g,
        test_node_ids,
        sampler,
        batch_size=16,
        shuffle=True,
        drop_last=False,
    )

In [100]:
sampler = dgl.dataloading.MultiLayerFullNeighborSampler(1)
trial_dataloader = dgl.dataloading.DataLoader(
        g, g.nodes(), 
        sampler, shuffle=True, drop_last=False,
        batch_size=16
        )

In [105]:
sum_values = 0
for input_nodes, output_nodes, blocks in trial_dataloader:
    sum_values += (len(blocks[0].srcdata['_ID'].tolist()))

print(sum_values)

684789


In [90]:
sum_values = 0
for pos_g, neg_g, blocks in trial_dataloader:
#     lst = pos_g.ndata['_ID'].tolist()
    sum_values += (len(pos_g.tolist()))

print(sum_values)

681635


In [79]:
g.number_of_edges()

1000209

In [68]:
681184+423304

1104488

In [25]:
valid_dataloader = dgl.dataloading.EdgeDataLoader(
        test_g, test_g.nodes(), 
        sampler, negative_sampler=dgl.dataloading.negative_sampler.Uniform(5), 
        shuffle=True, drop_last=False,
        batch_size=16
        )

In [15]:
def regress(model, bg, node_subgraph_negative):
    
    bg = bg.to('cpu')
    h = bg.ndata.pop('age')
    e = bg.edata.pop('rating')
    h, e = h.to('cpu', dtype=torch.float), e.to('cpu', dtype=torch.float)

    return model(bg, h, e, node_subgraph_negative)

In [16]:
from model import MPNNPredictor

In [17]:
def run_a_train_epoch_full(epoch, model, dataloader, optimizer):
    
    total_loss = 0
    batch_number = 0
    
    for _, pos_g, neg_g, blocks in dataloader:
        pos_score, neg_score = regress(model, pos_g, neg_g)
        loss = compute_loss(pos_score, neg_score)

        if epoch > 0:  # For the epoch 0, no training (just report loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        total_loss += loss.item()
        batch_number += 1

    train_avg_loss = total_loss / batch_number
    
    return total_loss

In [18]:
model = MPNNPredictor(node_in_feats=1,edge_in_feats=1,node_out_feats=64,edge_hidden_feats=128,n_tasks=12)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0001,weight_decay=0)

for epoch in range(2):
    average_loss = run_a_train_epoch_full(epoch, model, dataloader, optimizer)
    
    print("Epoch :{} has a average loss of : {}".format(epoch, average_loss))

    model.eval()
    with torch.no_grad():
        total_val_loss = 0
        i = 0
        for _, pos_g, neg_g, blocks in valid_dataloader:

            pos_score, neg_score = regress(model, pos_g, neg_g)
            loss = compute_loss(pos_score, neg_score)
            total_val_loss += loss.item()
            i += 1

        val_avg_loss = total_val_loss / i

    print("This model has a average validation loss of : {}".format(total_val_loss))

Epoch :0 has a average loss of : 416.3966514468193
This model has a average validation loss of : 417.83248245716095
Epoch :1 has a average loss of : 8.866619752370752
This model has a average validation loss of : 0.9992181684356183


In [39]:
df_final[df_final['user_id'].isin([3, 6])]

,user_id,movie_id,rating,timestamp,title,genre,gender,age,occupation,zipcode
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975),Drama,F,1,10,48067
1,1,661,3,978302109,James and the Giant Peach (1996),Animation|Children's|Musical,F,1,10,48067
2,1,914,3,978301968,My Fair Lady (1964),Musical|Romance,F,1,10,48067
3,1,3408,4,978300275,Erin Brockovich (2000),Drama,F,1,10,48067
4,1,2355,5,978824291,"Bug's Life, A (1998)",Animation|Children's|Comedy,F,1,10,48067
...,...,...,...,...,...,...,...,...,...,...
1000204,6040,1091,1,956716541,Weekend at Bernie's (1989),Comedy,M,25,6,11106
1000205,6040,1094,5,956704887,"Crying Game, The (1992)",Drama|Romance|War,M,25,6,11106
1000206,6040,562,5,956704746,Welcome to the Dollhouse (1995),Comedy|Drama,M,25,6,11106
1000207,6040,1096,4,956715648,Sophie's Choice (1982),Drama,M,25,6,11106


In [50]:
# code to get trained embedding for each node
h_item_batches = []

for _, pos_g, neg_g, blocks in valid_dataloader:
    lst = pos_g.ndata['_ID'].tolist()
    for value in lst:
        if value > 6040:
            print('Here')
#     print(df_final.query('movie_id in @lst'))
#     break
    h_item_batches.append(model.get_embeddings(pos_g))

[201, 96, 351, 376, 122, 248, 222, 43, 224, 156, 342, 14, 191, 243, 1865, 2207, 3005, 2352, 907, 3060, 1407, 890, 2107, 1475, 2377, 1460, 2881, 100, 1474, 1543, 880, 5359, 1562, 3244, 5623, 1045, 4272, 1919, 3944, 5068, 2744, 2310, 3099, 5003, 33, 382, 2398, 397, 5747, 4527, 266, 1491, 1262, 1079, 5887, 3682, 476, 2061, 2152, 1503, 2617, 1389, 3016, 2914, 4756, 235, 732, 4216, 3579, 3032, 1484, 1386, 5138, 2776, 4707, 3371, 1707, 1054, 3204, 3329, 353, 4123, 3802, 3347, 3561, 1134, 283, 4374, 1591, 412, 2482, 2654, 5032, 2777, 740, 4672, 1721, 2056, 624, 2724, 1754, 2462, 2889, 3741, 4578, 3172, 2643, 5067, 4601, 5950]
[306, 277, 211, 326, 91, 193, 5, 201, 348, 330, 194, 236, 122, 239, 174, 1232, 2616, 1941, 1038, 3081, 2723, 1946, 706, 17, 299, 2018, 33, 919, 3097, 2290, 4879, 1917, 5179, 673, 1145, 456, 3891, 1989, 5526, 5013, 5990, 467, 5199, 1707, 2799, 1183, 5298, 5789, 5029, 2006, 4243, 4372, 4259, 1350, 604, 4894, 3366, 948, 5384, 2063, 4581, 5531, 3070, 3468, 2557, 5968, 1722, 

[183, 240, 411, 342, 330, 57, 180, 96, 351, 9, 301, 12, 194, 115, 343, 274, 2075, 1283, 2061, 88, 1916, 2290, 69, 953, 2333, 1275, 2149, 1721, 3762, 2682, 3365, 2351, 3303, 3310, 2610, 1307, 3381, 4173, 3211, 372, 3544, 2023, 5703, 2638, 3861, 354, 3215, 1467, 3679, 5504, 3949, 3206, 3060, 818, 4497, 4853, 4859, 4337, 3207, 2995, 2022, 5964, 1152, 1975, 676, 659, 236, 311, 1812, 2672, 5425, 3080, 523, 4469, 3286, 2268, 1530, 4506, 4046, 5251, 2285, 1089, 2098, 4483, 4004, 620, 565, 580, 3529, 1671, 1860, 4541, 2366, 4897, 2430, 447, 1924, 5978, 908, 181, 1888, 1484, 1863, 3309, 1518, 1968, 1163, 5051, 5614, 1737, 4102]
[203, 149, 104, 356, 201, 53, 76, 404, 279, 244, 172, 267, 328, 330, 52, 1089, 594, 2946, 1747, 2004, 2790, 2958, 3146, 1306, 1473, 1967, 1592, 1499, 317, 192, 912, 1241, 947, 249, 4034, 3076, 1234, 5572, 372, 4551, 420, 1006, 5581, 5826, 3455, 797, 5506, 2290, 5345, 2433, 4052, 2548, 327, 3345, 5204, 4579, 5994, 3012, 2442, 5260, 5590, 1496, 1726, 1745, 666, 3526, 1788,

[135, 357, 388, 101, 241, 354, 116, 122, 23, 165, 203, 264, 328, 52, 267, 201, 3259, 913, 439, 2211, 3242, 159, 1286, 1089, 2114, 152, 1206, 1239, 2099, 3099, 3535, 5880, 4606, 65, 3933, 5085, 550, 5294, 2380, 4867, 3878, 4033, 4238, 2464, 399, 1475, 5291, 13, 3035, 1076, 5007, 2137, 4241, 5703, 2343, 1018, 2536, 3654, 112, 3079, 4998, 2059, 4434, 1279, 1164, 1032, 5632, 5508, 2052, 1338, 5578, 5126, 3950, 5659, 3032, 3767, 1710, 5559, 3012, 4585, 1447, 1501, 565, 3912, 1435, 2683, 4217, 1, 479, 4830, 4102, 1693, 5207, 4260, 155, 3473, 549, 5948, 3985, 1054, 2864, 163, 610, 2852, 5676, 4045, 5672, 3166, 5314, 4791]
[352, 201, 271, 318, 328, 61, 262, 209, 111, 338, 404, 16, 88, 384, 244, 191, 607, 1290, 33, 533, 1257, 3705, 1583, 1800, 587, 2619, 377, 588, 1217, 3264, 2370, 224, 3278, 2199, 5342, 381, 89, 1159, 4339, 2440, 1990, 4982, 4979, 4503, 5591, 4216, 2409, 605, 2482, 409, 5553, 1331, 223, 4897, 5877, 2298, 1199, 3673, 5416, 727, 5256, 3643, 1023, 3808, 5489, 5208, 2753, 4365, 38

[244, 201, 286, 194, 61, 207, 259, 301, 389, 258, 219, 214, 7, 342, 1590, 2121, 2395, 2656, 2123, 2915, 180, 2457, 1281, 911, 2624, 2989, 2916, 1209, 3385, 338, 3664, 3018, 2490, 5958, 1397, 2389, 856, 1761, 1945, 1014, 1409, 3431, 2142, 4570, 1025, 1193, 1678, 5901, 3952, 5106, 6012, 5152, 384, 3085, 5491, 186, 5072, 184, 5458, 139, 4067, 2460, 4864, 3461, 5178, 5433, 5682, 195, 3885, 131, 2928, 267, 4812, 2650, 4206, 3170, 1030, 406, 2508, 1698, 1922, 5520, 3860, 1148, 1182, 666, 2415, 3151, 1255, 463, 4919, 606, 2194, 2590, 973, 252, 2382, 5990, 4979, 5951, 3433, 5779, 899, 4313, 3199, 3667, 2408, 2480, 347, 4819]
[224, 165, 125, 121, 320, 211, 52, 234, 407, 215, 330, 172, 120, 302, 410, 997, 2010, 2019, 2408, 69, 343, 696, 3107, 1370, 646, 3504, 1198, 3698, 1390, 1996, 3821, 1559, 2906, 1157, 2429, 3162, 3970, 3827, 3792, 4617, 3003, 5705, 5776, 3240, 3920, 3952, 4260, 3975, 5448, 933, 4482, 3965, 2869, 3261, 1028, 34, 5835, 2500, 1989, 1258, 784, 1802, 2831, 884, 3337, 184, 1125, 

[197, 237, 307, 164, 339, 294, 354, 116, 351, 244, 216, 25, 150, 24, 325, 328, 3437, 1119, 3254, 3069, 1269, 784, 409, 2327, 2423, 592, 3577, 1264, 2265, 2627, 3526, 44, 4922, 4805, 3321, 2306, 3189, 3794, 3927, 2351, 5666, 1867, 298, 960, 4168, 93, 2080, 2501, 125, 977, 2674, 1272, 2951, 3894, 1745, 2696, 1064, 3655, 1868, 816, 4497, 5362, 889, 2650, 4194, 5812, 3603, 2821, 5423, 2170, 2445, 5735, 4206, 2390, 2481, 1458, 4950, 524, 3593, 5496, 4415, 1084, 4715, 993, 53, 5013, 2416, 1115, 3090, 176, 3111, 1058, 3164, 2027, 2021, 5179, 2669, 336, 1764, 2283, 2365, 4669, 2968, 5899, 4160, 1320, 2713, 3610, 595]
[356, 204, 330, 411, 180, 34, 168, 197, 351, 82, 177, 322, 222, 328, 381, 2958, 1225, 2911, 673, 1407, 3648, 3915, 2990, 2766, 2682, 1274, 3070, 903, 760, 2, 2519, 6024, 5607, 5629, 3999, 5737, 3463, 5604, 4904, 2862, 4678, 1620, 2718, 1415, 4351, 4957, 5992, 3572, 3338, 584, 5096, 3973, 4779, 4153, 1452, 3322, 4307, 1260, 4767, 2386, 5157, 3202, 3242, 3388, 767, 2661, 3926, 3011,

...

[224, 283, 191, 33, 114, 258, 341, 100, 348, 271, 24, 52, 411, 304, 198, 148, 1059, 3703, 1369, 3268, 3408, 3104, 328, 2352, 3645, 1597, 3797, 592, 719, 345, 3505, 3826, 1789, 3805, 2950, 4539, 1897, 1447, 1034, 3664, 527, 4119, 1197, 3287, 5932, 1467, 3336, 5499, 3658, 201, 2719, 3086, 4483, 4012, 5437, 4875, 4239, 1039, 5658, 994, 5714, 3402, 6000, 68, 676, 3265, 28, 1100, 3238, 1646, 2606, 4068, 5721, 1363, 2798, 4194, 4452, 2185, 3916, 5076, 5114, 4414, 1763, 2063, 1241, 902, 5036, 5806, 3188, 5622, 3375, 4490, 4592, 2555, 1520, 3072, 1242, 3523, 1244, 3711, 3494, 5791, 1875, 1851, 1384, 3900, 2595, 3562, 5569, 1265]
[101, 207, 32, 401, 192, 332, 148, 61, 223, 117, 21, 269, 14, 81, 274, 263, 2840, 2788, 2921, 1221, 1647, 1258, 28, 224, 2803, 3551, 2986, 1782, 2706, 1624, 1035, 103, 1513, 2916, 2980, 2436, 3586, 5700, 1417, 204, 2363, 769, 4990, 4056, 2760, 4715, 4138, 5200, 4223, 36, 1406, 6000, 5436, 6009, 3518, 4179, 1078, 6019, 478, 3646, 5914, 4714, 968, 5330, 3079, 5563, 2789,

[304, 164, 72, 159, 198, 88, 51, 172, 8, 92, 116, 201, 22, 239, 17, 260, 1977, 2946, 591, 479, 3792, 369, 588, 2919, 3005, 2627, 673, 1456, 520, 3577, 2700, 1233, 5453, 4431, 2025, 5860, 4655, 5407, 4624, 2464, 5763, 5140, 5838, 5082, 248, 1334, 3564, 561, 5868, 394, 2844, 4868, 4841, 3972, 5467, 6010, 403, 3124, 2943, 1216, 2416, 4543, 1440, 5823, 4836, 1376, 4519, 4706, 5207, 3829, 2719, 915, 672, 100, 3024, 5988, 3104, 5294, 1936, 2565, 1755, 5748, 5069, 3872, 1468, 4993, 5279, 89, 4812, 644, 4134, 1066, 1607, 258, 3485, 3784, 2712, 4184, 5571, 3514, 1769, 4650, 5372, 3434, 35, 3812, 4195, 1312, 204, 717, 5198, 3463]
[68, 370, 191, 336, 61, 375, 304, 52, 244, 156, 172, 155, 127, 109, 410, 430, 670, 747, 1783, 1572, 2503, 1219, 2705, 2466, 1551, 925, 0, 3750, 2279, 1304, 2945, 2934, 1685, 5964, 4588, 4535, 5534, 78, 2767, 2269, 2611, 614, 4841, 4003, 4177, 1426, 1475, 5904, 813, 338, 4820, 524, 3863, 830, 5266, 4052, 5840, 3385, 2416, 4910, 4250, 5589, 1196, 6038, 5010, 3746, 2024, 5

[147, 162, 156, 44, 203, 133, 385, 396, 280, 328, 271, 60, 318, 110, 148, 1726, 3514, 1274, 904, 1437, 434, 1244, 1239, 2857, 3325, 2398, 761, 1377, 1196, 985, 1320, 4237, 534, 4500, 4982, 4685, 4682, 5996, 5823, 1624, 4310, 868, 1653, 5211, 5240, 1665, 5897, 4947, 2317, 4231, 2378, 5203, 4118, 1574, 5714, 5781, 2828, 4875, 4889, 1867, 3894, 5700, 4664, 2961, 3644, 667, 1157, 5876, 221, 922, 2853, 2353, 6033, 1729, 2008, 4660, 1914, 1411, 5061, 3381, 209, 69, 719, 4076, 4069, 5255, 3725, 1952, 1649, 1012, 5191, 515, 3723, 5264, 4979, 4709, 2866, 2498, 4328, 767, 2544, 2613, 4495, 5651, 5623, 4428, 4355, 2490, 3277, 1866]
[330, 119, 156, 89, 229, 300, 148, 150, 267, 116, 130, 376, 47, 337, 251, 3067, 1195, 3702, 3004, 2354, 2124, 1844, 480, 3896, 2431, 1252, 3542, 1083, 3482, 3, 3266, 4388, 5002, 2417, 2596, 463, 5304, 1589, 5560, 2940, 1274, 3192, 3654, 5506, 2943, 3129, 5705, 3928, 2044, 5737, 4938, 1660, 1642, 3363, 1347, 5768, 5634, 3230, 5502, 2302, 4209, 3333, 1732, 5191, 2175, 44

[91, 122, 96, 307, 218, 339, 328, 264, 52, 389, 61, 262, 208, 241, 238, 58, 2010, 1954, 958, 1607, 3004, 3545, 1956, 911, 2277, 3937, 1225, 1882, 2335, 1371, 3948, 356, 2404, 5299, 1614, 4125, 1739, 5122, 4632, 3882, 2296, 3121, 4655, 1514, 2544, 3634, 2871, 5031, 1775, 2039, 2224, 1162, 5924, 3386, 4801, 2186, 3252, 1036, 2765, 2359, 4548, 1023, 251, 4513, 882, 566, 6030, 1164, 1063, 3731, 758, 4927, 5319, 778, 4331, 4061, 3484, 2098, 4130, 5505, 3477, 3235, 5650, 4098, 2453, 3597, 3855, 2239, 199, 4094, 5472, 2910, 1831, 2595, 974, 1357, 1878, 1589, 3408, 2390, 5215, 2674, 988, 1218, 5595, 1532, 5940, 5593, 2580, 1014, 5143]
[307, 391, 165, 191, 337, 25, 142, 413, 381, 320, 281, 351, 91, 172, 234, 20, 2992, 779, 1710, 3365, 2013, 1958, 589, 1264, 1541, 2801, 2320, 2143, 1332, 2173, 2572, 3224, 4352, 826, 2324, 5537, 5145, 1976, 823, 2281, 2263, 932, 390, 1801, 4827, 2157, 5296, 4014, 4847, 2224, 4575, 3227, 6003, 4790, 5535, 2206, 1924, 3423, 2424, 2695, 5067, 4173, 3211, 5889, 3857,

[180, 125, 397, 122, 285, 333, 219, 328, 134, 365, 283, 264, 198, 407, 201, 542, 411, 20, 3333, 3385, 1943, 3797, 2320, 323, 2682, 3526, 3438, 918, 2281, 1920, 5650, 3049, 1530, 2814, 3443, 1435, 4181, 2783, 1413, 3070, 5190, 1622, 5970, 2917, 4772, 1235, 2378, 4583, 2198, 3335, 917, 832, 346, 5649, 1982, 1913, 1836, 3679, 2129, 4516, 2455, 3607, 4995, 4442, 576, 1964, 5878, 2539, 595, 324, 1356, 3750, 331, 2331, 5685, 1633, 5827, 463, 6038, 2442, 499, 5324, 2951, 2215, 2338, 440, 3208, 4998, 1076, 5507, 1962, 2843, 993, 172, 4689, 1560, 3719, 3234, 2173, 185, 1121, 46, 4684, 1607, 4912, 3804, 919, 4257, 1046, 3887]
[168, 43, 339, 80, 318, 71, 147, 172, 149, 35, 201, 142, 402, 261, 18, 1543, 2017, 1874, 1393, 2761, 1703, 2052, 110, 1448, 3255, 2615, 1872, 785, 2053, 1339, 4868, 2736, 3769, 3552, 2697, 4880, 5964, 3510, 3835, 1865, 973, 5412, 5831, 4060, 4642, 2170, 5839, 568, 519, 4093, 1093, 5723, 4566, 37, 1498, 288, 3627, 5378, 2884, 2392, 3210, 4526, 3040, 55, 4592, 4992, 1998, 530

In [29]:
h_item_batches = torch.cat(h_item_batches)

In [31]:
h_item_batches.shape

torch.Size([41623, 64])

In [31]:
state = {
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
        'loss': total_val_loss,
#         'item_embeddings': h_item,
        'batch_size': 16
            }
import os
model_dir = '/Users/pratikaher/DGL/models/'
model_fn = "{}_model_{}.pth".format('gnn_trained_model', 10)
torch.save(state, os.path.join(model_dir, model_fn))

In [ ]:


# for epoch in range(10):
    
# #     with torch.no_grad():
#     total_loss = 0
#     i = 0
#     for _, pos_g, neg_g, blocks in dataloader:
#         pos_score, neg_score = regress(model, pos_g, neg_g)
#         loss = compute_loss(pos_score, neg_score)

#         if epoch > 0:  # For the epoch 0, no training (just report loss)
#             optimizer.zero_grad()
#             loss.backward()
#             optimizer.step()
            
#         total_loss += loss.item()
#         i += 1

#     train_avg_loss = total_loss / batch_number
    
    